# FRAP Model Demonstration

## Mechanistically Interpretable FRAP Analysis

This notebook demonstrates the reaction-diffusion and mass-conserving FRAP models.

### Key Principles
1. **No black-box fitting**: All models are based on physical principles
2. **Mass conservation**: Explicitly tracked and enforced
3. **PDE integration**: Recovery curves from numerical integration, not empirical exponentials
4. **Parameter identifiability**: Explicitly tested and reported

### Model Equations

**Reaction-Diffusion:**
$$\frac{\partial F}{\partial t} = D \nabla^2 F - k_{on} F + k_{off} B$$
$$\frac{\partial B}{\partial t} = k_{on} F - k_{off} B$$

**Mass-Conserving RD:**
$$\int (c_{condensed} + c_{dilute}) \, dV = \text{constant}$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from frap_models import ReactionDiffusionModel, MassConservingRDModel

# Set up plotting
plt.style.use('seaborn-v0_8-paper')
%matplotlib inline

## 1. Basic Reaction-Diffusion Model

In [ ]:
# Initialize model
rd_model = ReactionDiffusionModel()

# Define geometry
geometry = {
    'shape': (100, 100),
    'spacing': 0.1,  # μm
    'total_concentration': 1.0,
    'bound_fraction': 0.5,
    'bleach_region': {
        'type': 'circular',
        'center': (5.0, 5.0),  # μm
        'radius': 1.0,  # μm
        'bleach_depth': 0.9
    }
}

# Define parameters
params = {
    'D': 5.0,      # μm²/s
    'k_on': 1.0,   # 1/s
    'k_off': 1.0,  # 1/s
    'bleach_depth': 0.9
}

# Time points
timepoints = np.array([0, 0.5, 1, 2, 5, 10, 20, 40, 60])

# Simulate recovery
print("Simulating FRAP recovery...")
recovery = rd_model.simulate(params, geometry, timepoints)

# Plot
plt.figure(figsize=(8, 6))
plt.plot(timepoints, recovery, 'o-', linewidth=2, markersize=8)
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Normalized Fluorescence', fontsize=12)
plt.title('FRAP Recovery: Reaction-Diffusion Model', fontsize=14)
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.1])
plt.tight_layout()
plt.show()

print(f"\nRecovery curve: {recovery}")

## 2. Parameter Sensitivity Analysis

### Varying Diffusion Coefficient

In [ ]:
# Test different diffusion coefficients
D_values = [1.0, 5.0, 20.0]
colors = ['blue', 'green', 'red']

plt.figure(figsize=(10, 6))

for D, color in zip(D_values, colors):
    params_test = params.copy()
    params_test['D'] = D
    
    recovery = rd_model.simulate(params_test, geometry, timepoints)
    plt.plot(timepoints, recovery, 'o-', color=color, 
             label=f'D = {D} μm²/s', linewidth=2, markersize=6)

plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Normalized Fluorescence', fontsize=12)
plt.title('Effect of Diffusion Coefficient on FRAP Recovery', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.1])
plt.tight_layout()
plt.show()

### Varying Binding Kinetics

In [ ]:
# Test different binding rates (keeping equilibrium constant)
k_values = [(0.5, 0.5), (2.0, 2.0), (10.0, 10.0)]

plt.figure(figsize=(10, 6))

for (k_on, k_off), color in zip(k_values, colors):
    params_test = params.copy()
    params_test['k_on'] = k_on
    params_test['k_off'] = k_off
    
    recovery = rd_model.simulate(params_test, geometry, timepoints)
    plt.plot(timepoints, recovery, 'o-', color=color,
             label=f'k_on = k_off = {k_on} s⁻¹', linewidth=2, markersize=6)

plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Normalized Fluorescence', fontsize=12)
plt.title('Effect of Binding Kinetics on FRAP Recovery', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.1])
plt.tight_layout()
plt.show()

## 3. Mass-Conserving Model for Condensates

In [ ]:
# Initialize mass-conserving model
mcrd_model = MassConservingRDModel()

# Geometry for condensate
condensate_geometry = geometry.copy()
condensate_geometry['condensed_fraction'] = 0.7

# Parameters
mcrd_params = {
    'D_dilute': 10.0,      # Fast diffusion in dilute phase
    'D_condensed': 0.5,    # Slow diffusion in condensate
    'k_in': 1.0,           # Condensation rate
    'k_out': 1.0,          # Dissolution rate
    'condensed_fraction': 0.7,
    'bleach_depth': 0.9
}

# Simulate
print("Simulating mass-conserving FRAP recovery...")
recovery_mcrd = mcrd_model.simulate(mcrd_params, condensate_geometry, timepoints)

# Plot comparison
plt.figure(figsize=(10, 6))
plt.plot(timepoints, recovery, 'o-', label='Standard RD', linewidth=2, markersize=6)
plt.plot(timepoints, recovery_mcrd, 's-', label='Mass-Conserving RD', 
         linewidth=2, markersize=6)
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Normalized Fluorescence', fontsize=12)
plt.title('Comparison: RD vs Mass-Conserving RD', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.1])
plt.tight_layout()
plt.show()

## 4. Geometry Scaling

Recovery should scale predictably with bleach radius (diffusion-limited regime).

In [ ]:
# Test different bleach radii
radii = [0.5, 1.0, 2.0]

plt.figure(figsize=(10, 6))

for radius, color in zip(radii, colors):
    geom_test = geometry.copy()
    geom_test['bleach_region'] = {
        'type': 'circular',
        'center': (5.0, 5.0),
        'radius': radius,
        'bleach_depth': 0.9
    }
    
    recovery = rd_model.simulate(params, geom_test, timepoints)
    plt.plot(timepoints, recovery, 'o-', color=color,
             label=f'Radius = {radius} μm', linewidth=2, markersize=6)

plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Normalized Fluorescence', fontsize=12)
plt.title('Geometry Scaling: Effect of Bleach Radius', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.1])
plt.tight_layout()
plt.show()

print("\nNote: Larger bleach radius leads to slower recovery (diffusion-limited).")

## 5. Identification of Recovery Regimes

In [ ]:
from frap_models.coalescence import estimate_recovery_regime

# Diffusion-limited regime
params_diff_limited = {
    'D': 0.5,      # Slow diffusion
    'k_on': 10.0,  # Fast binding
    'k_off': 10.0,
    'bleach_depth': 0.9
}

# Exchange-limited regime
params_exch_limited = {
    'D': 20.0,    # Fast diffusion
    'k_on': 0.5,  # Slow binding
    'k_off': 0.5,
    'bleach_depth': 0.9
}

# Identify regimes
regime_diff = estimate_recovery_regime(params_diff_limited, geometry)
regime_exch = estimate_recovery_regime(params_exch_limited, geometry)

print(f"Slow diffusion, fast exchange: {regime_diff}")
print(f"Fast diffusion, slow exchange: {regime_exch}")

# Simulate both
recovery_diff = rd_model.simulate(params_diff_limited, geometry, timepoints)
recovery_exch = rd_model.simulate(params_exch_limited, geometry, timepoints)

# Plot
plt.figure(figsize=(10, 6))
plt.plot(timepoints, recovery_diff, 'o-', label='Diffusion-limited', 
         linewidth=2, markersize=8)
plt.plot(timepoints, recovery_exch, 's-', label='Exchange-limited', 
         linewidth=2, markersize=8)
plt.xlabel('Time (s)', fontsize=12)
plt.ylabel('Normalized Fluorescence', fontsize=12)
plt.title('Recovery Regimes', fontsize=14)
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.ylim([0, 1.1])
plt.tight_layout()
plt.show()

## Summary

This notebook demonstrates:

1. **Mechanistic modeling**: Recovery curves from PDE integration
2. **Parameter sensitivity**: Clear effects of D, k_on, k_off
3. **Geometry scaling**: Predictable dependence on bleach radius
4. **Mass conservation**: Enforced in appropriate models
5. **Recovery regimes**: Diffusion vs exchange-limited

### Next Steps

- Fit experimental data using `frap_fitting` module
- Assess parameter identifiability
- Combine with SPT data (see FRAP_SPT_Joint_Fit.ipynb)